In [3]:
import json
import datetime

In [2]:
p = '''
    {
        "time": {
            "objecttype": "datetime",
            "value": "2018-10-21T09:14:15"
            },
        "message": "created this json string"
    }
'''

In [7]:
d = json.loads(p)
d

{'time': {'objecttype': 'datetime', 'value': '2018-10-21T09:14:15'},
 'message': 'created this json string'}

In [5]:
from datetime import datetime

for key, value in d.items():
    if (isinstance(value, dict) and 
        'objecttype' in value and 
        value['objecttype'] == 'datetime'):
        d[key] = datetime.strptime(value['value'], '%Y-%m-%dT%H:%M:%S')

In [6]:
d

{'time': datetime.datetime(2018, 10, 21, 9, 14, 15),
 'message': 'created this json string'}

In [8]:
j = '''
    {
        "cake": "yummy chocolate cake",
        "myShare": {
            "objecttype": "fraction",
            "numerator": 1,
            "denominator": 8
        }
    }
'''

In [10]:
n = json.loads(j)

In [11]:
from fractions import Fraction

for key, value in n.items():
    if (isinstance(value, dict) and
        'objecttype' in value and
        value['objecttype'] == 'fraction'):
        numerator = value['numerator']
        denominator = value['denominator']
        n[key] = Fraction(numerator, denominator)

In [12]:
n

{'cake': 'yummy chocolate cake', 'myShare': Fraction(1, 8)}

In [13]:
j = '''
    {
        "time": {
            "objecttype": "datetime",
            "value": "2018-10-21T09:14:15"
            },
        "message": "created this json string"
    }
'''

In [14]:
def custom_decoder(arg):
    if 'objecttype' in arg and arg['objecttype'] == 'datetime':
        return datetime.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
    else:
        return arg  # important, otherwise we lose anything that's not a date!

In [15]:
custom_decoder(dict(objecttype='datetime', value='2018-10-21T09:14:15'))

datetime.datetime(2018, 10, 21, 9, 14, 15)

In [16]:
j = '''
    {
        "times": {
            "created": {
                "objecttype": "datetime",
                "value": "2018-10-21T09:14:15"
                },
            "updated": {
                "objecttype": "datetime",
                "value": "2018-10-22T10:00:05"
                }
            },
        "message": "log message here..."
    }
'''

In [17]:
d = json.loads(j, object_hook=custom_decoder)

In [18]:
d

{'times': {'created': datetime.datetime(2018, 10, 21, 9, 14, 15),
  'updated': datetime.datetime(2018, 10, 22, 10, 0, 5)},
 'message': 'log message here...'}

We can also extend this custom decoder to include other structures (schemas). Let's add in our fraction decoder:

In [19]:
def custom_decoder(arg):
    ret_value = arg
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            ret_value = datetime.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            ret_value = Fraction(arg['numerator'], arg['denominator'])
    return ret_value

In [20]:
j = '''
    {
        "cake": "yummy chocolate cake",
        "myShare": {
            "objecttype": "fraction",
            "numerator": 1,
            "denominator": 8
        },
        "eaten": {
            "at": {
                "objecttype": "datetime",
                "value": "2018-10-21T21:30:00"
                },
            "time_taken": "30 seconds"
        }
    }
'''

In [21]:
d = json.loads(j, object_hook=custom_decoder)

In [22]:
d


{'cake': 'yummy chocolate cake',
 'myShare': Fraction(1, 8),
 'eaten': {'at': datetime.datetime(2018, 10, 21, 21, 30),
  'time_taken': '30 seconds'}}

In [23]:
def custom_decoder(arg):
    ret_value = arg
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            ret_value = datetime.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            ret_value = Fraction(arg['numerator'], arg['denominator'])
        elif arg['objecttype'] == 'person':
            ret_value = Person(arg['name'], arg['ssn'])
    return ret_value

In [24]:
class Person:
    def __init__(self, name, ssn):
        self.name = name
        self.ssn = ssn
        
    def __repr__(self):
        return f'Person(name={self.name}, ssn={self.ssn})'

In [25]:
d = json.loads(j, object_hook=custom_decoder)

Again note that in all cases, the received argument is the string read from the json string.

Finally we have the object_pairs_hook argument. It works similarly to the object_hook with two differences:

the argument is a list of 2-tuples - the first value is the key, the second is the value
the list is ordered in the same order as the keys in the json document.
Remember that the dictionary is not guaranteed to be ordered in the same order as the keys in the json document - given Python 3.6+ has guaranteed dictionary order, this is likely to be true, but the documents do not mention this specifically, so at this point it should be considered an implementation detail and not relied on - if you must have gauranteed key order, then you will have to use the object_pairs_hook.

Also, you should not specify both object_hook and object_pairs_hook - if you do, then the object_pairs_hook will be used and object_hook will be ignored.

In [26]:
j = '''
    {
        "a": [1, 2, 3, 4, 5],
        "b": 100,
        "c": 10.5,
        "d": NaN,
        "e": null,
        "f": "python"
    }
'''

In [27]:
def float_handler(arg):
    print('float handler', type(arg), arg)
    return float(arg)

In [28]:
def int_handler(arg):
    print('int handler', type(arg), arg)
    return int(arg)

In [29]:
def const_handler(arg):
    print('const handler', type(arg), arg)
    return None

In [30]:
def obj_hook(arg):
    print('obj hook', type(arg), arg)
    return arg

In [31]:
def obj_pairs_hook(arg):
    print('obj pairs hook', type(arg), arg)
    return arg

In [32]:
json.loads(j, 
           object_hook=obj_hook,
           parse_float=float_handler,
           parse_int=int_handler,
           parse_constant=const_handler
          )

int handler <class 'str'> 1
int handler <class 'str'> 2
int handler <class 'str'> 3
int handler <class 'str'> 4
int handler <class 'str'> 5
int handler <class 'str'> 100
float handler <class 'str'> 10.5
const handler <class 'str'> NaN
obj hook <class 'dict'> {'a': [1, 2, 3, 4, 5], 'b': 100, 'c': 10.5, 'd': None, 'e': None, 'f': 'python'}


{'a': [1, 2, 3, 4, 5],
 'b': 100,
 'c': 10.5,
 'd': None,
 'e': None,
 'f': 'python'}